In [32]:
print("ok")

ok


In [33]:
%pwd

'/Users/chaitanyagarg/Desktop/VS Code /Medical-Chatbot-'

In [34]:
import os 
os.chdir("../")

In [35]:
%pwd

'/Users/chaitanyagarg/Desktop/VS Code '

In [36]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [37]:
# Extra Data from PDF File 
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents

In [38]:
extracted_data = load_pdf_file(data='Data/')

In [39]:
# extracted_data

In [40]:
#Split the Data into Text Chunks

def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks 

In [41]:
text_chunks=text_split(extracted_data)
print("Legnth of Text Chunks :- ",len(text_chunks))

Legnth of Text Chunks :-  0


In [42]:
text_chunks

[]

In [43]:
from langchain_community.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    return embeddings


In [44]:
embeddings = download_hugging_face_embeddings()

In [45]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [46]:
# query_result

In [47]:
from dotenv import load_dotenv
load_dotenv()

False

In [48]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [49]:
from pinecone import Pinecone, ServerlessSpec
import os

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

index_name = "medicalbot"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print("Index created!")
else:
    print("Index already exists")


Index already exists


In [50]:
import os 
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [51]:
# Embed each chunk and upsert the embeddings into your Pinecone index.

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name
)

In [52]:
# Load Existing index

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embedding into Pinecone index 
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [53]:
docsearch

In [54]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [55]:
retrieved_docs = retriever.invoke("What is Acne?")

In [56]:
retrieved_docs

[Document(id='6a0b0a94-74a2-4bdb-a27a-20d4b3d49bb2', metadata={'author': 'Clifford', 'creationdate': '2004-12-28T15:38:25-05:00', 'creator': 'PyPDF', 'enhanced': 'By PDF Enhancer 2.5/Win', 'moddate': '2005-05-04T13:53:15-06:00', 'page': 4.0, 'page_label': '5', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data/Book.pdf', 'spdf': '1096', 'total_pages': 599.0}, page_content='A\nAbscess\nAcidophilus\nAcne\nAconite\nAcupressure\nAcupuncture\nAdemetionine\nAdie’s pupil\nAfrican pygeum\nAgastache\nAging\nAIDS\nAlcoholism\nAlexander technique\nAlfalfa\nAlisma\nAllergies\nAllium cepa\nAloe\nAlpha-hydroxy\nAlzheimer’s disease\nAmino acids\nAndrographis\nAndrostenedione\nAnemarrhena\nAnemia\nAngelica root\nAngina\nAnise\nAnkylosing spondylitis\nAnorexia nervosa\nAnthroposophical medicine\nAntioxidants\nAnxiety\nBates method\nBayberry\nBedsores\nBedwetting\nBee pollen\nBehavioral optometry\nBehavioral therapy'),
 Document(id='fbb3a83c-e7cf-425a-befb-052837b22af6', metadata={'author': 'Cliffo

In [57]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=150)

In [58]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you "
    "don't know. use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [59]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain= create_retrieval_chain(retriever, question_answer_chain)

In [60]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])

PermissionDeniedError: Error code: 403 - {'error': {'message': 'Project `proj_M7zC1FHxvdFmJxaX7ggsGWoL` does not have access to model `gpt-3.5-turbo-instruct`', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}